In [2]:
!pip install control
!pip install vitaldb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.1/455.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 6.4 MB/s eta 0:00:00


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 21 09:50:05 2022

@author: aubouinb
"""
import numpy as np
import control
from tqdm.notebook import tqdm

def PropoModel(model: str, age: int, sex: bool, weight: float, height: float):
    """PK model for propofol, with BIS and MAP effect sites.

    Parameters
    ----------
    model : str
        Specify the computation of the coefficient can be 'Schnider - Minto', 'Marsh - Minto' or 'Eleveld'.
    age : int
        Age of the patient in year.
    sex : Bool
        1=male, 0=female.
    weight : float
        weight in kg.
    height : float
        height in cm.

    Returns
    -------
    v1 : float
        Volume of the blood compartment.
    A : np.array
        matrix of the linear system x(k+1) = A x(k) + B u(k),
        where x includes Cblood, Cfat, Cmuscle, Cbis, Cmap1, Cmap2 in this order.

    """
    if model == 'Schnider - Minto':

        if sex == 1:  # homme
            lbm = 1.1 * weight - 128 * (weight / height) ** 2
        else:  # femme
            lbm = 1.07 * weight - 148 * (weight / height) ** 2

        # Clearance Rates [l/min]
        cl1 = 1.89 + 0.0456 * (weight - 77) - 0.0681 * (lbm - 59) + 0.0264 * (height - 177)
        cl2 = 1.29 - 0.024 * (age - 53)
        cl3 = 0.836
        # Volume of the compartments [l]
        v1 = 4.27
        v2 = 18.9 - 0.391 * (age - 53)
        v3 = 238
        # drug amount transfer rates [1/min]
        ke0 = 0.456
        k1e = ke0
    elif model == 'Marsh - Minto':

        # Volume of the compartments [l]
        v1 = 0.228 * weight
        v2 = 0.463 * weight
        v3 = 2.893 * weight
        # Clearance Rates [l/min]
        cl1 = 0.119 * v1
        cl2 = 0.112 * v1
        cl3 = 0.042 * v1
        # drug amount transfer rates [1/min]
        ke0 = 1.2
        k1e = ke0

    elif model == 'Eleveld':

        # see D. J. Eleveld, P. Colin, A. R. Absalom, and M. M. R. F. Struys,
        # “Pharmacokinetic–pharmacodynamic model for propofol for broad application in anaesthesia and sedation”
        # British Journal of Anaesthesia, vol. 120, no. 5, pp. 942–959, mai 2018, doi:10.1016/j.bja.2018.01.018.

        # reference patient
        AGE_ref = 35
        WGT_ref = 70
        HGT_ref = 1.7
        PMA_ref = (40+AGE_ref*52)/52  # not born prematurely and now 35 yo
        BMI_ref = WGT_ref/HGT_ref**2
        GDR_ref = 1  # 1 male, 0 female

        opiate = True  # consider opiate or not
        measurement = "arterial"  # can be "arterial" or "venous"
        theta = [None,                    # just to get same index than in the paper
                 6.2830780766822,       # V1ref [l]
                 25.5013145036879,      # V2ref [l]
                 272.8166615043603,     # V3ref [l]
                 1.7895836588902,       # Clref [l/min]
                 1.7500983738779,       # Q2ref [l/min]
                 1.1085424008536,       # Q3ref [l/min]
                 0.191307,              # Typical residual error
                 42.2760190602615,      # CL maturation E50
                 9.0548452392807,       # CL maturation slope [weeks]
                 -0.015633,             # Smaller V2 with age
                 -0.00285709,           # Lower CL with age
                 33.5531248778544,      # Weight for 50 % of maximal V1 [kg]
                 -0.0138166,            # Smaller V3 with age
                 68.2767978846832,      # Maturation of Q3 [weeks]
                 2.1002218877899,       # CLref (female) [l/min]
                 1.3042680471360,       # Higher Q2 for maturation of Q3
                 1.4189043652084,       # V1 venous samples (children)
                 0.6805003109141]       # Higer Q2 venous samples

        # function used in the model
        def faging(x): return np.exp(x * (age - AGE_ref))
        def fsig(x, C50, gam): return x**gam/(C50**gam + x**gam)
        def fcentral(x): return fsig(x, theta[12], 1)

        def fal_sallami(sexX, weightX, ageX, bmiX):
            if sexX:
                return (0.88 + (1-0.88)/(1+(ageX/13.4)**(-12.7)))*(9270*weightX)/(6680+216*bmiX)
            else:
                return (1.11 + (1 - 1.11)/(1+(ageX/7.1)**(-1.1)))*(9270*weightX)/(8780+244*bmiX)

        PMA = age + 40/52
        BMI = weight/(height/100)**2

        fCLmat = fsig(PMA * 52, theta[8], theta[9])
        fCLmat_ref = fsig(PMA_ref*52, theta[8], theta[9])
        fQ3mat = fsig(PMA * 52, theta[14], 1)
        fQ3mat_ref = fsig(PMA_ref * 52, theta[14], 1)
        fsal = fal_sallami(sex, weight, age, BMI)
        fsal_ref = fal_sallami(GDR_ref, WGT_ref, AGE_ref, BMI_ref)

        if opiate:
            def fopiate(x): return np.exp(x*age)
        else:
            def fopiate(x): return 1

        # reference: male, 70kg, 35 years and 170cm

        v1 = theta[1] * fcentral(weight)/fcentral(WGT_ref)
        if measurement == "venous":
            v1 = v1 * (1 + theta[17] * (1 - fcentral(weight)))
        v2 = theta[2] * weight/WGT_ref * faging(theta[10])
        v2ref = theta[2]
        v3 = theta[3] * fsal/fsal_ref * fopiate(theta[13])
        v3ref = theta[3]
        cl1 = (sex*theta[4] + (1-sex)*theta[15]) * (weight/WGT_ref)**0.75 * \
            fCLmat/fCLmat_ref * fopiate(theta[11])

        cl2 = theta[5]*(v2/v2ref)**0.75 * (1 + theta[16] * (1 - fQ3mat))
        if measurement == "venous":
            cl2 = cl2*theta[18]

        cl3 = theta[6] * (v3/v3ref)**0.75 * fQ3mat/fQ3mat_ref

        ke0 = 0.146*(weight/70)**(-0.25)
        k1e = ke0

    # MAP effect site transfert rates
    ke0_1 = 0.0540
    ke0_2 = 0.0695
    # drug amount transfer rates [1/min]
    k10 = cl1 / v1
    k12 = cl2 / v1
    k13 = cl3 / v1
    k21 = cl2 / v2
    k31 = cl3 / v3

    # Matrices system definition
    A = np.array([[-(k10 + k12 + k13), k21, k31, 0, 0, 0],
                  [k12, -k21, 0, 0, 0, 0],
                  [k13, 0, -k31, 0, 0, 0],
                  [k1e, 0, 0, -ke0, 0, 0],
                  [ke0_1, 0, 0, 0, -ke0_1, 0],
                  [ke0_2, 0, 0, 0, 0, -ke0_2]])

    return v1, A


def RemiModel(model: str, age: int, sex: bool, weight: float, height: float):
    """PK model for remifentanil, with BIS and MAP effect sites.

    Parameters
    ----------
    model : str
        Specify the computation of the coefficient can be 'Schnider - Minto', 'Marsh - Minto' or 'Eleveld'.
    age : int
        Age of the patient in year.
    sex : Bool
        1=male, 0=female.
    weight : float
        weight in kg.
    height : float
        height in cm.

    Returns
    -------
    v1 : float
        Volume of the blood compartment.
    A : np.array
        matrix of the linear system x(k+1) = A x(k) + B u(k),
        where x includes Cblood, Cfat, Cmuscle, Cbis, Cmap in this order.

    """
    if model == 'Marsh - Minto' or model == 'Schnider - Minto':
        if sex == 1:  # homme
            lbm = 1.1 * weight - 128 * (weight / height) ** 2
        else:  # femme
            lbm = 1.07 * weight - 148 * (weight / height) ** 2

        # Clearance Rates [l/min]
        cl1 = 2.6 + 0.0162 * (age - 40) + 0.0191 * (lbm - 55)
        cl2 = 2.05 - 0.0301 * (age - 40)
        cl3 = 0.076 - 0.00113 * (age - 40)
        # Volume of the compartments [l]
        v1 = 5.1 - 0.0201 * (age - 40) + 0.072 * (lbm - 55)
        v2 = 9.82 - 0.0811 * (age - 40) + 0.108 * (lbm - 55)
        v3 = 5.42
        # drug amount transfer rates [1/min]
        ke0 = 0.595 - 0.007 * (age - 40)
        k1e = ke0  # 0.456
    if model == 'Eleveld':
        # see D. J. Eleveld et al., “An Allometric Model of Remifentanil Pharmacokinetics and Pharmacodynamics,”
        # Anesthesiology, vol. 126, no. 6, pp. 1005–1018, juin 2017, doi: 10.1097/ALN.0000000000001634.

        # function used in the model
        def faging(x): return np.exp(x * (age - 35))
        def fsig(x, C50, gam): return x**gam/(C50**gam + x**gam)

        def fal_sallami(sexX, weightX, ageX, bmiX):
            if sexX:
                return (0.88 + (1-0.88)/(1+(ageX/13.4)**(-12.7)))*(9270*weightX)/(6680+216*bmiX)
            else:
                return (1.11 + (1 - 1.11)/(1+(ageX/7.1)**(-1.1)))*(9270*weightX)/(8780+244*bmiX)

        # reference patient
        AGE_ref = 35
        WGT_ref = 70
        HGT_ref = 1.7
        BMI_ref = WGT_ref/HGT_ref**2
        GDR_ref = 1  # 1 male, 0 female

        BMI = weight/(height/100)**2

        SIZE = (fal_sallami(sex, weight, age, BMI)/fal_sallami(GDR_ref, WGT_ref, AGE_ref, BMI_ref))

        theta = [None,      # Juste to have the same index as in the paper
                 2.88,
                 -0.00554,
                 -0.00327,
                 -0.0315,
                 0.470,
                 -0.0260]

        KMAT = fsig(weight, theta[1], 2)
        KMATref = fsig(WGT_ref, theta[1], 2)
        if sex:
            KSEX = 1
        else:
            KSEX = 1+theta[5]*fsig(age, 12, 6)*(1-fsig(age, 45, 6))

        v1ref = 5.81
        v1 = v1ref * SIZE * faging(theta[2])
        V2ref = 8.882
        v2 = V2ref * SIZE * faging(theta[3])
        V3ref = 5.03
        v3 = V3ref * SIZE * faging(theta[4])*np.exp(theta[6]*(weight - WGT_ref))
        cl1ref = 2.58
        cl2ref = 1.72
        cl3ref = 0.124
        cl1 = cl1ref * SIZE**0.75 * (KMAT/KMATref)*KSEX*faging(theta[3])
        cl2 = cl2ref * (v2/V2ref)**0.75 * faging(theta[2]) * KSEX
        cl3 = cl3ref * (v3/V3ref)**0.75 * faging(theta[2])
        ke0 = 1.09 * faging(-0.0289)
        k1e = ke0

    ke0_MAP = 0.81
    # drug amount transfer rates [1/min]
    k10 = cl1 / v1
    k12 = cl2 / v1
    k13 = cl3 / v1
    k21 = cl2 / v2
    k31 = cl3 / v3

    # Matrices system definition
    A = np.array([[-(k10 + k12 + k13), k21, k31, 0, 0],
                  [k12, -k21, 0, 0, 0],
                  [k13, 0, -k31, 0, 0],
                  [k1e, 0, 0, -ke0, 0],
                  [ke0_MAP, 0, 0, 0, -ke0_MAP]])

    return v1, A


def surface_model(x_p: list, x_r: list, base_MAP: float):
    """
    Surface model for Propofol-Remifentanil interaction on BIS and MAP.

    Coefficients come from:
        - "Refining Target-Controlled Infusion: An Assessment of Pharmacodynamic Target-Controlled Infusion of
            Propofol and Remifentanil Using a Response Surface Model of Their Combined Effects on Bispectral Index"
            by Short et al.
        - "Pharmacokinetic–pharmacodynamic modeling of the hypotensive effect of remifentanil in infants undergoing
            cranioplasty" by Standing et al.
        - "Pharmacodynamic response modelling of arterial blood pressure in adult volunteers during propofol
            anaesthesia" by C. Jeleazcov et al.

    Parameters
    ----------
    x_p : list
        state vector of Propofol PK model.
    x_r : list
        state vector of remifentanil Pk model.
    base_MAP : float
        Initial MAP.

    Returns
    -------
    bis : float
        in (%).
    Map : float
        in mmHg.

    """
    # BIS computation
    c50p_bis = 4.47
    C50r_bis = 19.3
    beta = 0
    gamma = 1.43
    E0 = 97.4

    cep_BIS = x_p[3]
    cer_BIS = x_r[3]
    up = cep_BIS / c50p_bis
    ur = cer_BIS / C50r_bis
    if (up+ur) != 0:
        Phi = up/(up + ur)
    else:
        Phi = 0
    U_50 = 1 - beta * (Phi - Phi**2)
    i = (up + ur)/U_50
    bis = E0 - E0 * i ** gamma / (1 + i ** gamma)

    # MAP computation
    # propofol influence
    Emax_SAP = 54.8
    Emax_DAP = 18.1
    EC50_1 = 1.96
    gamma_1 = 4.77
    EC50_2 = 2.20
    gamma_2 = 8.49

    U = (x_p[4]/EC50_1)**gamma_1 + (x_p[5]/EC50_2)**gamma_2
    Effect_Propo = - (Emax_DAP + (Emax_SAP+Emax_DAP)/3) * U/(1+U)

    # Remifenatnil influence
    EC50 = 17.1
    gamma = 4.56
    Emax = 69.7

    Effect_remi = - Emax * (x_p[4]**gamma/(x_p[4]**gamma + EC50**gamma))

    Map = base_MAP + Effect_Propo + Effect_remi

    return bis, Map


def discretize(A: list, B: list, Ts: float = 1):
    """Discretize LTI system.

    Parameters
    ----------
    A : list
        Dynamic matrix of the system dx/dt = Ax + Bu.
    B : list
        Input matrix of the system dx/dt = Ax + Bu.
    Ts : float
        Sampling time (s). Default is 1s.

    Returns
    -------
    Ad : list
        Dynamic matrix of the system x(t+Te) = Ad x(t) + Bd u(t).
    Bd : list
        Input matrix of the system x(t+Te) = Ad x(t) + Bd u(t).

    """
    """Discretize the system dx/dt = Ax + Bu and return Ad and Bd such that x(t+Te) = Ad x(t) + Bd u(t),
        Te is given in (s)"""
    C = np.zeros((1, len(A)))
    D = 0
    model = control.ss(np.array(A)/60, B, C, D)  # A divided by 60 because it was in 1/min
    model = control.sample_system(model, Ts)
    Ad = model.A
    Bd = model.B
    return Ad, Bd

# Create dataset

In [4]:
"""Create a simple dataset from vitalDB."""


import pandas as pd
import numpy as np
import vitaldb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
from tqdm.notebook import tqdm
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


plt.rc('text', usetex=True)
plt.rc('font', family='serif')


def formate_patient_data(data):
    """Format the data extracted from the .csv file to be able to read them as integers."""
    print("Formating Patient Data............................")
    for s in tqdm(['age', 'height', 'weight']):
        if s == 'age':
            data[s] = data[s].str.replace(r'.', '0', regex=False)
        else:
            data[s] = data[s].str.replace(r'. ', '0', regex=False)
        data[s] = data[s].str.replace(r' %', '0', regex=False)
        data[s] = data[s].str.replace(r'0%', '0', regex=False)
        data[s] = data[s].astype(float)
    return data


def load_cases(track_names: list, caseids: list):
    """Import a list of cases from vitaldb in a dataframe format."""
    print("Loading Cases............................")
    dataframe_final = pd.DataFrame()
    for caseid in tqdm(caseids):
        cases = vitaldb.VitalFile(caseid, track_names)
        dataframe_temp = cases.to_pandas(track_names, 1)
        dataframe_temp.insert(0, 'caseid', caseid)
        dataframe_final = pd.concat([dataframe_final, dataframe_temp], ignore_index=True)
    return dataframe_final



# %% Load data
perso_data = pd.read_csv("https://api.vitaldb.net/cases")
#perso_data = formate_patient_data(perso_data)


# id_list


caselist = [46, 70, 101, 167, 172, 218, 221, 247, 268, 345, 353, 405, 447, 533, 537, 544, 545, 585,
            593, 636, 663, 671, 672, 685, 711, 734, 751, 812, 827, 831, 835, 847, 866, 872, 894,
            926, 940, 952, 963, 1029, 1044, 1047, 1154, 1176, 1201, 1212, 1213, 1237, 1238, 1239, 1267,
            1376, 1392, 1396, 1398, 1404, 1416, 1440, 1602, 1611, 1613, 1657, 1658, 1662,
            1687, 1690, 1918, 1925, 1994, 2000, 2014, 2029, 2049, 2051, 2072, 2074, 2139, 2148,
            2157, 2196, 2229, 2238, 2309, 2379, 2382, 2392, 2409, 2442, 2479, 2480, 2500, 2511,
            2527, 2528, 2542, 2562, 2569, 2572, 2949, 2955, 2956, 2975, 3027, 3042, 3047, 3050,
            3065, 3070, 3073, 3092, 3315, 3366, 3367, 3376, 3379, 3398, 3407, 3435, 3458, 3710, 3729,
            3791, 3859, 4050, 4091, 4098, 4122, 4146, 4172, 4173, 4177, 4195, 4202, 4212, 4253,
            4277, 4292, 4350, 4375, 4387, 4432, 4472, 4547, 4673, 4678, 4716, 4741, 4745, 4789, 4803]  # 4768


id_train, id_test = train_test_split(caselist, test_size=0.3, random_state=4)  # split between test and train
id_train = np.random.permutation(id_train)
id_split = np.array_split(id_train, 3)  # split train set in 5 set for cross validation


# import the cases
cases = load_cases(['BIS/BIS', 'Orchestra/PPF20_RATE', 'Orchestra/RFTN20_RATE',
                    'Orchestra/PPF20_CE', 'Orchestra/RFTN20_CE', 'Solar8000/ART_MBP',
                    'BIS/SQI', 'Solar8000/PLETH_HR', 'Orchestra/PPF20_CP',
                    'Orchestra/RFTN20_CP', 'Orchestra/RFTN20_VOL',
                    'Orchestra/PPF20_VOL', 'Solar8000/NIBP_MBP'], caseids=caselist)  # load the case from vitalDB

cases.rename(columns={'BIS/BIS': 'BIS',
                      'Orchestra/PPF20_RATE': 'Propofol',
                      'Orchestra/RFTN20_RATE': "Remifentanil",
                      'Orchestra/PPF20_CE': "Ce_Prop",
                      'Orchestra/RFTN20_CE': "Ce_Rem",
                      'Solar8000/ART_MBP': "MAP",
                      'BIS/SQI': "SQI",
                      'Solar8000/PLETH_HR': "HR",
                      'Orchestra/PPF20_CP': "Cp_Prop",
                      'Orchestra/RFTN20_CP': "Cp_Rem",
                      'Orchestra/RFTN20_VOL': 'Vol_Rem',
                      'Orchestra/PPF20_VOL': 'Vol_Prop',
                      'Solar8000/NIBP_MBP': 'NI_MAP'}, inplace=True)

# define bound for the values
cols = ['BIS', 'MAP', 'HR', 'Propofol', 'Remifentanil', "Ce_Prop",
        "Ce_Rem", "SQI", 'age', 'sex', 'height', 'weight', 'bmi']

min_val = {'BIS': 10, 'MAP': 50, 'Propofol': 0, 'Remifentanil': 0, "Ce_Prop": 0, "Ce_Rem": 0, "SQI": 50}
max_val = {'BIS': 100, 'MAP': 160, 'Propofol': 1e3, 'Remifentanil': 1e3, "Ce_Prop": 1e3, "Ce_Rem": 1e3, "SQI": 100}

# %%
Patients_train = pd.DataFrame()
Patients_test = pd.DataFrame()

nb_points = 0
hist_Cp = 10*60
windows_Cp = 30
win_vec = np.ones(windows_Cp)

Loading Cases............................


  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
print("Patients_train & Patients_test............................")
for caseid in tqdm(cases['caseid'].unique()):
    print(caseid)
    Patient_df = cases[cases['caseid'] == caseid].reset_index(drop=True)
    Patient_df = Patient_df.copy()

    # find MAP baseline
    Map_base_case = Patient_df['NI_MAP'].fillna(method='bfill')[0]
    Patient_df.insert(len(Patient_df.columns), "MAP_base_case", Map_base_case)

    # compute median HR
    median_window = 600
    Patient_df.loc[:, 'mean_HR'] = Patient_df.loc[:, 'HR'].rolling(
        median_window, min_periods=1, center=False).apply(np.nanmedian)

    # replace nan by 0 in drug rates
    Patient_df['Propofol'].fillna(method='bfill', inplace=True)
    Patient_df['Remifentanil'].fillna(method='bfill', inplace=True)

    # find first drug injection
    print("# find first drug injection")
    istart = 0
    for i in tqdm(range(len(Patient_df))):
        if Patient_df.loc[i, 'Propofol'] != 0 or Patient_df.loc[i, 'Remifentanil'] != 0:
            istart = i
            break
    # removed before strating of anesthesia
    print("# removed before strating of anesthesia")
    Patient_df = Patient_df[istart:]
    Patient_df.reset_index(inplace=True)

    Patient_df['BIS'].replace(0, np.nan, inplace=True)
    Patient_df['MAP'].replace(0, np.nan, inplace=True)
    Patient_df['HR'].replace(0, np.nan, inplace=True)

    # remove artefact in map measure
    print("# remove artefact in map measure")
    Patient_df.loc[abs(Patient_df['MAP']-np.nanmean(Patient_df['MAP'].values)) > 50, 'MAP'] = np.nan * \
        np.ones((len(Patient_df.loc[abs(Patient_df['MAP']-np.nanmean(Patient_df['MAP'].values)) > 50, 'MAP'])))

    # remove bad quality point for BIS
    print("# remove bad quality point for BIS")
    Patient_df.loc[Patient_df['SQI'] < 50, 'BIS'] = np.nan * \
        np.ones((len(Patient_df.loc[Patient_df['SQI'] < 50, 'BIS'])))

    window_size = 30  # Mean window

    # fig, ax = plt.subplots()
    # Patient_df['BIS'].plot(ax = ax)

    # fig2, ax2 = plt.subplots()
    # Patient_df.loc[1000:1500,'BIS'].plot(ax = ax2)

    L = Patient_df['BIS'].to_numpy()
    for i in range(len(L)):
        if not np.isnan(L[i]):
            i_first_non_nan = i
            break

    L = np.concatenate((Patient_df.loc[i_first_non_nan, 'BIS']*np.ones(500), L))
    L = pd.DataFrame(L)
    L = L.ewm(span=20, min_periods=1).mean()

    Patient_df.loc[:, 'BIS'] = L[500:].to_numpy()

    Patient_df.loc[:, 'MAP'] = Patient_df['MAP'].ewm(span=20, min_periods=1).mean()

    # Patient_df.loc[1000:1500,'BIS'].plot(ax = ax2)
    # plt.title('case = ' + str(caseid))
    # plt.show()

    # Patient_df['BIS'].plot(ax = ax)
    # plt.title('case = ' + str(caseid))
    # plt.show()

    Patient_df.loc[:, 'HR'] = Patient_df['HR'].rolling(window_size, min_periods=1, center=True).apply(np.nanmean)

    Patient_df = Patient_df.fillna(method='ffill')

    Patient_df.insert(len(Patient_df.columns), "full_BIS", 0)
    Patient_df.insert(len(Patient_df.columns), "full_MAP", 0)

    Patient_df.loc[(Patient_df['BIS'] <= min_val['BIS']) | (Patient_df['BIS'] >= max_val['BIS']), 'full_BIS'] = np.ones(
        (len(Patient_df.loc[(Patient_df['BIS'] <= min_val['BIS']) | (Patient_df['BIS'] >= max_val['BIS']), 'full_BIS'])))

    Patient_df.loc[(Patient_df['MAP'] <= min_val['MAP']) | (Patient_df['MAP'] >= max_val['MAP']), 'full_MAP'] = np.ones(
        (len(Patient_df.loc[(Patient_df['MAP'] <= min_val['MAP']) | (Patient_df['MAP'] >= max_val['MAP']), 'full_MAP'])))

    Patient_df.loc[Patient_df['BIS'].isna(), 'full_BIS'] = np.ones(
        (len(Patient_df.loc[Patient_df['BIS'].isna(), 'full_BIS'])))
    Patient_df.loc[Patient_df['MAP'].isna(), 'full_MAP'] = np.ones(
        (len(Patient_df.loc[Patient_df['MAP'].isna(), 'full_MAP'])))

    Patient_df.insert(len(Patient_df.columns), "med_BIS", np.nan)
    Patient_df.insert(len(Patient_df.columns), "med_MAP", np.nan)

    Patient_df.loc[:, 'med_BIS'] = Patient_df.loc[:, 'BIS'].rolling(median_window, center=False).median()
    Patient_df.loc[:, 'med_MAP'] = Patient_df.loc[:, 'MAP'].rolling(median_window, center=False).median()

    # Patient_df.insert(len(Patient_df.columns),"mean_HR", np.nanmedian(Patient_df.loc[:15*60, 'HR']))

    # find first MAP non Nan
    # for i in range(len(Patient_df)):
    #     if not np.isnan(Patient_df.loc[i,"MAP"]):
    #         first_map = i
    #         break
    # # find first BIS non Nan
    # for i in range(len(Patient_df)):
    #     if not np.isnan(Patient_df.loc[i,"BIS"]):
    #         first_bis = i
    #         break
    # median_window = 600

    # Patient_df.insert(len(Patient_df.columns),"med_BIS", np.nanmedian(Patient_df.loc[first_bis:median_window + first_bis,'BIS']))
    # Patient_df.insert(len(Patient_df.columns),"med_MAP", np.nanmedian(Patient_df.loc[first_map :median_window + first_map,'MAP']))

    # Patient_df.loc[:median_window + first_bis,'med_BIS'] = np.nan*np.ones(median_window + first_bis + 1)
    # Patient_df.loc[:median_window + first_map,'med_MAP'] = np.nan*np.ones(median_window + first_map + 1)

    nb_points += len(Patient_df['BIS'])
    Patient_df.insert(1, "Time", np.arange(0, len(Patient_df['BIS'])))
    age = float(perso_data[perso_data['caseid'] == caseid]['age'])
    Patient_df.insert(len(Patient_df.columns), "age", age)
    sex = int(perso_data[perso_data['caseid'] == caseid]['sex'] == 'M')  # F = 0, M = 1
    Patient_df.insert(len(Patient_df.columns), "sex", sex)
    weight = float(perso_data[perso_data['caseid'] == caseid]['weight'])
    Patient_df.insert(len(Patient_df.columns), "weight", weight)
    height = float(perso_data[perso_data['caseid'] == caseid]['height'])
    Patient_df.insert(len(Patient_df.columns), "height", height)
    Patient_df.insert(len(Patient_df.columns), "bmi", float(perso_data[perso_data['caseid'] == caseid]['bmi']))

    if sex == 1:  # homme
        lbm = 1.1 * weight - 128 * (weight / height) ** 2
    else:  # femme
        lbm = 1.07 * weight - 148 * (weight / height) ** 2
    Patient_df.insert(len(Patient_df.columns), "lbm", lbm)

    model = "Eleveld"

    v1_p, Ap = PropoModel(model, age, sex, weight, height)
    v1_r, Ar = RemiModel(model, age, sex, weight, height)

    Bp = np.zeros((6, 1))
    Bp[0, 0] = 1 / v1_p
    Br = np.zeros((5, 1))
    Br[0, 0] = 1 / v1_r

    Adp, Bdp = discretize(Ap, Bp, 1)
    Adr, Bdr = discretize(Ar, Br, 1)

    x_p = np.zeros((6, 1))
    x_r = np.zeros((5, 1))
    Patient_df.insert(len(Patient_df.columns), "Ce_Prop_MAP", 0)
    Patient_df.insert(len(Patient_df.columns), "Ce_Rem_MAP", 0)

    Ncase = len(Patient_df['BIS'])
    Cp_Prop = np.zeros(Ncase)
    Cp_Rem = np.zeros(Ncase)
    Ce_Prop = np.zeros(Ncase)
    Ce_Rem = np.zeros(Ncase)
    Ce_Prop_MAP = np.zeros(Ncase)
    Ce_Rem_MAP = np.zeros(Ncase)
    print("Propofol & Remifentanil")
    for j in range(Ncase-1):
        x_p = Adp @ x_p + Bdp * Patient_df['Propofol'][j]*20/3600
        x_r = Adr @ x_r + Bdr * Patient_df['Remifentanil'][j]*20/3600

        Cp_Prop[j+1] = x_p[0]
        Cp_Rem[j+1] = x_r[0]
        Ce_Prop[j+1] = x_p[3]
        Ce_Rem[j+1] = x_r[3]
        Ce_Prop_MAP[j+1] = (x_p[4] + x_p[5])/2
        Ce_Rem_MAP[j+1] = x_r[4]

    Patient_df["Cp_Prop_Eleveld"] = Cp_Prop
    Patient_df["Cp_Rem_Eleveld"] = Cp_Rem
    Patient_df["Ce_Prop_Eleveld"] = Ce_Prop
    Patient_df["Ce_Rem_Eleveld"] = Ce_Rem
    Patient_df["Ce_Prop_MAP_Eleveld"] = Ce_Prop_MAP
    Patient_df["Ce_Rem_MAP_Eleveld"] = Ce_Rem_MAP

    if caseid in id_test:
        Patients_test = pd.concat([Patients_test, Patient_df], ignore_index=True)
    else:
        for i in range(len(id_split)):
            if caseid in id_split[i]:
                set_int = i
                break
        Patient_df.insert(len(Patient_df.columns), "train_set", set_int)
        Patients_train = pd.concat([Patients_train, Patient_df], ignore_index=True)
    print("_____________________________________________________________________________________________________")
print("Train & Test separation Complete")


# Print stats
print("nb point tot: " + str(nb_points))
print("nb point train: " + str(len(Patients_train['BIS'])))
print("nb point test: " + str(len(Patients_test['BIS'])))


print_dist = True

Patients_train.to_csv("/content/drive/MyDrive/Iqram Sir/PK_PD/Patients_train.csv",index=False)
Patients_test.to_csv("/content/drive/MyDrive/Iqram Sir/PK_PD/Patients_test.csv", index=False)

In [6]:
Patients_train.to_csv("/content/drive/MyDrive/Iqram Sir/PK_PD/Patients_train.csv",index=False)
Patients_test.to_csv("/content/drive/MyDrive/Iqram Sir/PK_PD/Patients_test.csv", index=False)

In [ ]:
"""


# Save Patients DataFrame
# Patients_train.to_csv("./Patients_train.csv")
# Patients_test.to_csv("./Patients_test.csv")


# %%
if print_dist:

    Data_train = []
    Data_test = []
    print("Train")
    for case in tqdm(id_train):
        p = Patients_train[Patients_train['caseid'] == case]
        temp = p[['age', 'height', 'weight', 'sex']].to_numpy()
        Data_train.append(temp[0])

    print("Test")
    for case in tqdm(id_test):
        p = Patients_test[Patients_test['caseid'] == case]
        temp = p[['age', 'height', 'weight', 'sex']].to_numpy()
        Data_test.append(temp[0])

    Data_plot_train = pd.DataFrame(data=Data_train, columns=['age', 'height', 'weight', 'sex'])
    Data_plot_test = pd.DataFrame(data=Data_test, columns=['age', 'height', 'weight', 'sex'])

    fig, axs = plt.subplots(2, 2)
    Data_plot_test['age'].hist(label="test", ax=axs[0, 0])
    Data_plot_train['age'].hist(label="train", alpha=0.5, ax=axs[0, 0])
    axs[0, 0].set_title('Age')
    axs[0, 0].legend()

    Data_plot_test['weight'].hist(label="test", ax=axs[0, 1])
    Data_plot_train['weight'].hist(label="train", alpha=0.5, ax=axs[0, 1])
    axs[0, 1].set_title('Weight')

    Data_plot_test['height'].hist(label="test", ax=axs[1, 0])
    Data_plot_train['height'].hist(label="train", alpha=0.5, ax=axs[1, 0])
    axs[1, 0].set_title('Height')

    data = []
    for i in range(len(Data_plot_test['sex'])):
        data.append(('M'*int(Data_plot_test['sex'][i]) + 'F'*(1 - int(Data_plot_test['sex'][i]))))
    print_df = pd.DataFrame(data, columns=['sex'])
    print_df['sex'].hist(label="test", ax=axs[1, 1])
    data = []
    for i in range(len(Data_plot_train['sex'])):
        data.append(('M'*int(Data_plot_train['sex'][i]) + 'F'*(1 - int(Data_plot_train['sex'][i]))))
    print_df = pd.DataFrame(data, columns=['sex'])
    print_df['sex'].hist(label="train", alpha=0.5, ax=axs[1, 1])
    axs[1, 1].set_title('Sex')
    fig.tight_layout()
    savepath = "dataset_info.pdf"
    #fig.savefig(savepath, bbox_inches='tight', format='pdf')
    plt.show()"""

#Matrics Functions

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 23 10:09:38 2022

@author: aubouinb
"""
import numpy as np
import pandas as pd
import torch
import bokeh
import matplotlib
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.models import Range1d
from bokeh.io import export_svg
from matplotlib import pyplot as plt
from matplotlib import cm
from sklearn.preprocessing import PolynomialFeatures

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

plt.rc('text', usetex=True)
plt.rc('font', family='serif')


def compute_metrics(data):
    """compute the metrics MDPE +/- SD, MDAPE +/- SD and RMSE +/- SD for a prediction signals
    over a patient population.
    Inputs:     - data is a panda dataframe with the fiels case_id, true_*, pred_*
    Output:     -  caseid of the best and worst RMSE
    print also the results to copy them in a latex table"""

    MDPE = 0
    MDAPE = 0
    RMSE = 0

    SD_MDPE = 0
    SD_MDAPE = 0

    RMSE_BIS_list = []
    RMSE_list = []
    case_length_list = []
    rmse_max = -1
    rmse_min = 1e10

    for col_name in data.columns:
        if 'true' in col_name:
            true_col = col_name
        elif 'pred' in col_name:
            pred_col = col_name
    print(pred_col)
    print(true_col)

    for case in data['case_id'].unique():
        case_data = data[data['case_id'] == case]
        case_length = len(case_data)
        PE = 100 * (case_data[true_col].values - case_data[pred_col].values)/case_data[true_col].values

        MDPE += case_length * np.median(PE)

        MDAPE += case_length * np.median(np.abs(PE))

        efficiency_case = 4*int(case_length/2)/(np.pi * case_length)

        SD_MDPE += case_length * np.var(PE) / efficiency_case
        SD_MDAPE += case_length * np.var(np.abs(PE)) / efficiency_case

        rmse = np.sqrt(np.mean((case_data[true_col].values - case_data[pred_col].values)**2))

        RMSE += case_length * rmse

        RMSE_list.append(rmse)
        case_length_list.append(case_length)

        if rmse > rmse_max:
            case_rmse_max = case
            rmse_max = rmse
        if rmse < rmse_min:
            case_rmse_min = case
            rmse_min = rmse
    sample_nb = len(data)

    MDPE /= sample_nb
    MDAPE /= sample_nb
    RMSE /= sample_nb

    SD_MDPE = np.sqrt(SD_MDPE / sample_nb)
    SD_MDAPE = np.sqrt(SD_MDAPE / sample_nb)

    SD_RMSE = np.sqrt(np.sum([(RMSE_list[i] - RMSE)**2 * case_length_list[i]
                      for i in range(len(RMSE_list))]) / sample_nb)

    col_name = pred_col[5:]
    print("                 ______   "+col_name+" results   ______")
    print("     MDPE \t & \t MDAPE \t & \t RMSE       ")

    print("$" + str(round(MDPE, 1)) + " \pm " + str(round(SD_MDPE, 1))
          + "$ & $" + str(round(MDAPE, 1)) + " \pm " + str(round(SD_MDAPE, 1))
          + "$ & $" + str(round(RMSE, 1)) + " \pm " + str(round(SD_RMSE, 1)) + "$")

    return case_rmse_max, case_rmse_min


def plot_results(data_BIS, data_MAP, data_train_BIS=pd.DataFrame(), data_train_MAP=pd.DataFrame()):
    """plot the result of the prediction with bokeh module"""
    y_true_test = data_BIS["true_BIS"].values
    y_pred_test = data_BIS["pred_BIS"].values
    if not data_train_BIS.empty:
        train = True
        y_true_train = data_train_BIS["true_BIS"].values
        y_pred_train = data_train_BIS["pred_BIS"].values
    else:
        train = False

    fig1 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
    fig1.line(range(0, len(y_true_test)), y_true_test, line_color='navy', legend_label="y")
    fig1.line(range(0, len(y_pred_test)), y_pred_test, line_color='red', legend_label="y predicted")
    fig1.xaxis.axis_label = "time [s]"
    fig1.yaxis.axis_label = "y"

    fig2 = figure(plot_width=900, plot_height=450, title="Fitting error")
    fig2.line(range(0, len(y_true_test)), y_true_test-y_pred_test, line_color='navy', legend_label="y")
    fig2.xaxis.axis_label = "time [s]"
    fig2.yaxis.axis_label = "y"

    fig3 = figure(plot_width=900, plot_height=450, title="BIS")
    fig3.circle(y_true_test, y_pred_test, legend_label='y', size=2, color="navy", alpha=0.1)
    if train:
        fig3.circle(y_true_train, y_pred_train, legend_label='train', size=2, color="green", alpha=0.1)

    fig3.line(np.array([y_true_test.min(), y_true_test.max()]), np.array([y_true_test.min(), y_true_test.max()]),
              line_color="red")
    fig3.legend.location = "top_left"
    fig3.xaxis.axis_label = "y measured"
    fig3.yaxis.axis_label = "y predicted"
    fig3.x_range = Range1d(y_true_test.min(), y_true_test.max())
    fig3.y_range = Range1d(y_true_test.min(), y_true_test.max())

    fig4 = figure(plot_width=900, plot_height=450, title="Trained fit")
    if train:
        fig4 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
        fig4.line(range(0, len(y_true_train)), y_true_train, line_color='navy', legend_label="y traint rue")
        fig4.line(range(0, len(y_pred_train)), y_pred_train, line_color='red', legend_label="y train predicted")
        fig4.xaxis.axis_label = "time [s]"
        fig4.yaxis.axis_label = "y"

    layout = row(column(fig1, fig2), column(fig3, fig4))
    show(layout)

    y_true_test = data_MAP["true_MAP"].values
    y_pred_test = data_MAP["pred_MAP"].values
    if train:
        y_true_train = data_train_MAP["true_MAP"].values
        y_pred_train = data_train_MAP["pred_MAP"].values

    fig1 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
    fig1.line(range(0, len(y_true_test)), y_true_test, line_color='navy', legend_label="y")
    fig1.line(range(0, len(y_pred_test)), y_pred_test, line_color='red', legend_label="y predicted")
    fig1.xaxis.axis_label = "time [s]"
    fig1.yaxis.axis_label = "y"

    fig2 = figure(plot_width=900, plot_height=450, title="Fitting error")
    fig2.line(range(0, len(y_true_test)), y_true_test-y_pred_test, line_color='navy', legend_label="y")
    fig2.xaxis.axis_label = "time [s]"
    fig2.yaxis.axis_label = "y"

    fig3 = figure(plot_width=900, plot_height=450, title="MAP")
    fig3.circle(y_true_test, y_pred_test, legend_label='y', size=2, color="navy", alpha=0.1)
    if train:
        fig3.circle(y_true_train, y_pred_train, legend_label='train', size=2, color="green", alpha=0.1)

    fig3.line(np.array([y_true_test.min(), y_true_test.max()]), np.array([y_true_test.min(), y_true_test.max()]),
              line_color="red")
    fig3.legend.location = "top_left"
    fig3.xaxis.axis_label = "y measured"
    fig3.yaxis.axis_label = "y predicted"
    fig3.x_range = Range1d(y_true_test.min(), y_true_test.max())
    fig3.y_range = Range1d(y_true_test.min(), y_true_test.max())

    fig4 = figure(plot_width=900, plot_height=450, title="Trained fit")
    if train:
        fig4 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
        fig4.line(range(0, len(y_true_train)), y_true_train, line_color='navy', legend_label="y traint rue")
        fig4.line(range(0, len(y_pred_train)), y_pred_train, line_color='red', legend_label="y train predicted")
        fig4.xaxis.axis_label = "time [s]"
        fig4.yaxis.axis_label = "MAP (mmHg)"

    layout = row(column(fig1, fig2), column(fig3, fig4))
    show(layout)


def plot_one_fig(case_full, case_pred, output, columns_pred, columns_pred_full):

    color = list(bokeh.palettes.Category10[max(3, len(columns_pred)+1+len(columns_pred_full))])

    fig = figure(plot_width=500, plot_height=250)
    i = 0
    fig.line(case_full['Time'].values/60, case_full[output].values,
             line_color=color[i], legend_label="True " + output, line_width=3)

    for name in columns_pred_full:
        i += 1
        fig.line(case_full['Time'].values/60, case_full[name], legend_label=name[9:], line_color=color[i], line_width=2)

    for name in columns_pred:
        i += 1
        fig.line(case_pred['Time'].values/60, case_pred[name], legend_label=name[9:], line_color=color[i], line_width=2)

    fig.xaxis.axis_label = "time (min)"
    fig.yaxis.axis_label = output + '(%)'*(output == 'BIS') + '(mmHg)'*(output == 'MAP')
    return fig


def plot_case(Patient_pred_BIS, Patient_pred_MAP, Patient_full, caseid_min_bis, case_min_map, caseid_max_bis, caseid_max_map):
    # case minimum
    case_pred = Patient_pred_BIS[Patient_pred_BIS['caseid'] == caseid_min_bis]
    case_full = Patient_full[Patient_full['caseid'] == caseid_min_bis]

    columns_pred_BIS = [name for name in case_pred.columns if name[:8] == 'pred_BIS']
    columns_pred_BIS_full = [name for name in case_full.columns if name[:8] == 'pred_BIS']

    fig1 = plot_one_fig(case_full, case_pred, 'BIS', columns_pred_BIS, columns_pred_BIS_full)
    fig1.title.text = 'BIS best case'

    case_pred = Patient_pred_MAP[Patient_pred_MAP['caseid'] == case_min_map]
    case_full = Patient_full[Patient_full['caseid'] == case_min_map]

    columns_pred_MAP = [name for name in case_pred.columns if name[:8] == 'pred_MAP']
    columns_pred_MAP_full = [name for name in case_full.columns if name[:8] == 'pred_MAP']

    fig2 = plot_one_fig(case_full, case_pred, 'MAP', columns_pred_MAP, columns_pred_MAP_full)
    fig2.title.text = 'MAP best case'

    case_pred = Patient_pred_BIS[Patient_pred_BIS['caseid'] == caseid_max_bis]
    case_full = Patient_full[Patient_full['caseid'] == caseid_max_bis]

    fig3 = plot_one_fig(case_full, case_pred, 'BIS', columns_pred_BIS, columns_pred_BIS_full)
    fig3.title.text = 'BIS worst case'

    case_pred = Patient_pred_MAP[Patient_pred_MAP['caseid'] == caseid_max_map]
    case_full = Patient_full[Patient_full['caseid'] == caseid_max_map]

    fig4 = plot_one_fig(case_full, case_pred, 'MAP', columns_pred_MAP, columns_pred_MAP_full)
    fig4.title.text = 'MAP worst case'

    layout = row(column(fig1, fig2), column(fig3, fig4))
    show(layout)
    fig1.output_backend = "svg"
    export_svg(fig1, filename="fig1.svg")
    fig2.output_backend = "svg"
    export_svg(fig2, filename="fig2.svg")
    fig3.output_backend = "svg"
    export_svg(fig3, filename="fig3.svg")
    fig4.output_backend = "svg"
    export_svg(fig4, filename="fig4.svg")


def plot_surface(reg, scaler, feature, pca=None):
    """Plot the 3D surface of the BIS related to Propofol and Remifentanil effect site concentration"""
    age = 35
    weight = 70
    height = 170
    sex = 1
    bmi = weight / (height/100)**2
    if sex == 1:  # homme
        lbm = 1.1 * weight - 128 * (weight / height) ** 2
    else:  # femme
        lbm = 1.07 * weight - 148 * (weight / height) ** 2
    MAP_base = 100
    HR_base = 80
    cer = np.linspace(1, 6, 50)
    cep = np.linspace(1, 6, 50)
    output = np.zeros((50, 50))
    X_p = np.zeros((50, 50))
    Y_r = np.zeros((50, 50))
    for i in range(len(cep)):
        for j in range(len(cer)):
            if feature == '-Cplasma' or feature == '-Cmap' or feature == '-Cbis':
                input = np.array([age, sex, height, weight, bmi, lbm, HR_base,
                                 cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-hr':
                input = np.array([age, sex, height, weight, bmi, lbm, cep[i], cer[j],
                                 cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == 'All':
                input = np.array([age, sex, height, weight, bmi, lbm, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-bmi':
                input = np.array([age, sex, height, weight, lbm, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-lbm':
                input = np.array([age, sex, height, weight, bmi, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-map':
                input = np.array([age, sex, height, weight, bmi, lbm, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            input = PolynomialFeatures(degree=1, include_bias=False).fit_transform(input)
            input = scaler.transform(input)
            # input = pca.transform(input)
            # input = input[:,:20]
            output[i, j] = reg.predict(input)
            X_p[i, j] = cep[i]
            Y_r[i, j] = cer[j]
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    surf = ax.plot_surface(X_p, Y_r, output, cmap=cm.jet, linewidth=0.1)
    ax.set_xlabel('$C_{Remifentanil}$')
    ax.set_ylabel('$C_{Propofol}$')
    ax.set_zlabel('BIS')
    fig.colorbar(surf, shrink=0.5, aspect=20)
    ax.view_init(30, 17)
    savepath = "surf1.pdf"
    fig.savefig(savepath, bbox_inches='tight', format='pdf')
    plt.show()


def plot_surface_tensor(reg, scaler, feature, pca=None):
    """Plot the 3D surface of the BIS related to Propofol and Remifentanil effect site concentration"""
    age = 35
    weight = 70
    height = 170
    sex = 1
    bmi = weight / (height/100)**2
    if sex == 1:  # homme
        lbm = 1.1 * weight - 128 * (weight / height) ** 2
    else:  # femme
        lbm = 1.07 * weight - 148 * (weight / height) ** 2
    MAP_base = 100
    HR_base = 80
    cer = np.linspace(1, 6, 50)
    cep = np.linspace(1, 6, 50)
    output = np.zeros((50, 50))
    X_p = np.zeros((50, 50))
    Y_r = np.zeros((50, 50))
    for i in range(len(cep)):
        for j in range(len(cer)):
            if feature == '-Cplasma' or feature == '-Cmap' or feature == '-Cbis':
                input = np.array([age, sex, height, weight, bmi, lbm, HR_base,
                                 cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-hr':
                input = np.array([age, sex, height, weight, bmi, lbm, cep[i], cer[j],
                                 cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == 'All':
                input = np.array([age, sex, height, weight, bmi, lbm, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-bmi':
                input = np.array([age, sex, height, weight, lbm, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-lbm':
                input = np.array([age, sex, height, weight, bmi, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            if feature == '-map':
                input = np.array([age, sex, height, weight, bmi, lbm, HR_base, cep[i],
                                 cer[j], cep[i], cer[j], cep[i], cer[j]]).reshape(1, -1)
            input = PolynomialFeatures(degree=1, include_bias=False).fit_transform(input)
            input = scaler.transform(input)
            # input = pca.transform(input)
            # input = input[:,:20]
            input = torch.tensor(input.astype(np.float32))
            output[i, j] = reg.predict(input)
            X_p[i, j] = cep[i]
            Y_r[i, j] = cer[j]
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    surf = ax.plot_surface(X_p, Y_r, (output+1)*50, cmap=cm.jet, linewidth=0.1)
    ax.set_xlabel('$C_{Remifentanil}$')
    ax.set_ylabel('$C_{Propofol}$')
    ax.set_zlabel('BIS')
    fig.colorbar(surf, shrink=0.5, aspect=20)
    ax.view_init(30, 17)
    savepath = "surf1.pdf"
    fig.savefig(savepath, bbox_inches='tight', format='pdf')
    plt.show()


def plot_dresults(data_BIS, data_MAP, data_train_BIS=pd.DataFrame(), data_train_MAP=pd.DataFrame()):
    """plot the result of the prediction with bokeh module"""
    y_true_test = data_BIS["true_dBIS"].values
    y_pred_test = data_BIS["pred_dBIS"].values
    if not data_train_BIS.empty:
        train = True
        y_true_train = data_train_BIS["true_dBIS"].values
        y_pred_train = data_train_BIS["pred_dBIS"].values
    else:
        train = False

    fig1 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
    fig1.line(range(0, len(y_true_test)), y_true_test, line_color='navy', legend_label="y")
    fig1.line(range(0, len(y_pred_test)), y_pred_test, line_color='red', legend_label="y predicted")
    fig1.xaxis.axis_label = "time [s]"
    fig1.yaxis.axis_label = "y"

    fig2 = figure(plot_width=900, plot_height=450, title="Fitting error")
    fig2.line(range(0, len(y_true_test)), y_true_test-y_pred_test, line_color='navy', legend_label="y")
    fig2.xaxis.axis_label = "time [s]"
    fig2.yaxis.axis_label = "y"

    fig3 = figure(plot_width=900, plot_height=450, title="BIS")
    fig3.circle(y_true_test, y_pred_test, legend_label='y', size=2, color="navy", alpha=0.1)
    if train:
        fig3.circle(y_true_train, y_pred_train, legend_label='train', size=2, color="green", alpha=0.1)

    fig3.line(np.array([y_true_test.min(), y_true_test.max()]), np.array([y_true_test.min(), y_true_test.max()]),
              line_color="red")
    fig3.legend.location = "top_left"
    fig3.xaxis.axis_label = "y measured"
    fig3.yaxis.axis_label = "y predicted"
    fig3.x_range = Range1d(y_true_test.min(), y_true_test.max())
    fig3.y_range = Range1d(y_true_test.min(), y_true_test.max())

    fig4 = figure(plot_width=900, plot_height=450, title="Trained fit")
    if train:
        fig4 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
        fig4.line(range(0, len(y_true_train)), y_true_train, line_color='navy', legend_label="y traint rue")
        fig4.line(range(0, len(y_pred_train)), y_pred_train, line_color='red', legend_label="y train predicted")
        fig4.xaxis.axis_label = "time [s]"
        fig4.yaxis.axis_label = "y"

    layout = row(column(fig1, fig2), column(fig3, fig4))
    show(layout)

    y_true_test = data_MAP["true_dMAP"].values
    y_pred_test = data_MAP["pred_dMAP"].values
    if train:
        y_true_train = data_train_MAP["true_dMAP"].values
        y_pred_train = data_train_MAP["pred_dMAP"].values

    fig1 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
    fig1.line(range(0, len(y_true_test)), y_true_test, line_color='navy', legend_label="y")
    fig1.line(range(0, len(y_pred_test)), y_pred_test, line_color='red', legend_label="y predicted")
    fig1.xaxis.axis_label = "time [s]"
    fig1.yaxis.axis_label = "y"

    fig2 = figure(plot_width=900, plot_height=450, title="Fitting error")
    fig2.line(range(0, len(y_true_test)), y_true_test-y_pred_test, line_color='navy', legend_label="y")
    fig2.xaxis.axis_label = "time [s]"
    fig2.yaxis.axis_label = "y"

    fig3 = figure(plot_width=900, plot_height=450, title="MAP")
    fig3.circle(y_true_test, y_pred_test, legend_label='y', size=2, color="navy", alpha=0.1)
    if train:
        fig3.circle(y_true_train, y_pred_train, legend_label='train', size=2, color="green", alpha=0.1)

    fig3.line(np.array([y_true_test.min(), y_true_test.max()]), np.array([y_true_test.min(), y_true_test.max()]),
              line_color="red")
    fig3.legend.location = "top_left"
    fig3.xaxis.axis_label = "y measured"
    fig3.yaxis.axis_label = "y predicted"
    fig3.x_range = Range1d(y_true_test.min(), y_true_test.max())
    fig3.y_range = Range1d(y_true_test.min(), y_true_test.max())

    fig4 = figure(plot_width=900, plot_height=450, title="Trained fit")
    if train:
        fig4 = figure(plot_width=900, plot_height=450, title="Data (blue) Vs Fitted data (red)")
        fig4.line(range(0, len(y_true_train)), y_true_train, line_color='navy', legend_label="y traint rue")
        fig4.line(range(0, len(y_pred_train)), y_pred_train, line_color='red', legend_label="y train predicted")
        fig4.xaxis.axis_label = "time [s]"
        fig4.yaxis.axis_label = "MAP (mmHg)"

    layout = row(column(fig1, fig2), column(fig3, fig4))
    show(layout)

#Standard model

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 20 12:46:15 2022

@author: aubouinb
"""

import numpy as np
import pandas as pd
#from metrics_functions import compute_metrics
import matplotlib.pyplot as plt
#import model

# Model to test

model_name = 'Eleveld'
# model_name = 'Schnider - Minto'
# model_name = 'Marsh - Minto'


# %% load data
Patients_test = pd.read_csv("Patients_test.csv", index_col=0)

# %% Perform simulation

Output_df = pd.DataFrame(columns=['case_id', 'true_BIS', 'pred_BIS', 'true_MAP', 'pred_MAP', 'full_BIS', 'full_MAP'])

i = 0
if 'pred_BIS_' + model_name not in Patients_test.columns:
    Patients_test.insert(len(Patients_test.columns), 'pred_BIS_'+model_name, 0)

for caseid in tqdm(Patients_test['caseid'].unique()):
    print(caseid)

    Patient_df = Patients_test[Patients_test['caseid'] == caseid].copy().reset_index()
    Output_df_temp = pd.DataFrame(columns=['case_id', 'true_BIS', 'pred_BIS',
                                  'true_MAP', 'pred_MAP', 'full_BIS', 'full_MAP'])

    Patient_df['MAP'] = Patient_df['MAP'].fillna(0)

    # create model
    MAP_base_case = Patient_df['MAP_base_case'][0]
    age = int(Patient_df['age'][0])
    height = int(Patient_df['height'][0])
    weight = int(Patient_df['weight'][0])
    sex = int(Patient_df['sex'][0])

    v1_p, Ap = PropoModel(model_name, age, sex, weight, height)
    v1_r, Ar = RemiModel(model_name, age, sex, weight, height)

    Bp = np.zeros((6, 1))
    Bp[0, 0] = 1 / v1_p
    Br = np.zeros((5, 1))
    Br[0, 0] = 1 / v1_r

    Adp, Bdp = discretize(Ap, Bp, 1)
    Adr, Bdr = discretize(Ar, Br, 1)

    Ncase = len(Patient_df['BIS'])
    Output_df_temp['true_BIS'] = Patient_df['BIS']
    Output_df_temp['true_MAP'] = Patient_df['MAP']
    Output_df_temp['full_BIS'] = Patient_df['full_BIS']
    Output_df_temp['full_MAP'] = Patient_df['full_MAP']
    Output_df_temp['case_id'] = np.ones((Ncase)) * caseid

    Output_df_temp['pred_BIS'] = np.zeros((Ncase))
    Output_df_temp['pred_MAP'] = np.zeros((Ncase))

    x_p = np.zeros((6, 1))
    x_r = np.zeros((5, 1))
    Output_df_temp.loc[0, 'pred_BIS'], Output_df_temp.loc[0, 'pred_MAP'] = surface_model(x_p, x_r, MAP_base_case)
    for j in range(Ncase-1):
        x_p = Adp @ x_p + Bdp * Patient_df['Propofol'][j]*20/3600
        x_r = Adr @ x_r + Bdr * Patient_df['Remifentanil'][j]*20/3600

        Bis, Map = surface_model(x_p, x_r, MAP_base_case)
        Output_df_temp.loc[j+1, 'pred_BIS'] = Bis
        Output_df_temp.loc[j+1, 'pred_MAP'] = Map

    Output_df = pd.concat([Output_df, Output_df_temp], ignore_index=True)

    # Output_df_temp['pred_BIS'] = Output_df_temp['pred_BIS'].diff()
    # Output_df_temp['true_BIS'] = Output_df_temp['true_BIS'].diff()
    # Output_df_temp.loc[0, 'true_BIS'] = 0
    # Output_df_temp.loc[0, 'pred_BIS'] = 0
    Patients_test.loc[Patients_test["caseid"] == caseid, 'pred_BIS_'+model_name] = Output_df_temp['pred_BIS'].values
    Patients_test.loc[Patients_test["caseid"] == caseid, 'pred_MAP_'+model_name] = Output_df_temp['pred_MAP'].values

    """if i % 5 == 0:
        fig, ax = plt.subplots(2)
        ax[1].set_xlabel('caseid : ' + str(caseid))
        ax[0].plot(Patient_df['BIS'])
        ax[0].plot(Output_df_temp['pred_BIS'])
        ax[1].plot(Patient_df['MAP'])
        ax[1].plot(Output_df_temp['pred_MAP'])
        plt.pause(0.05)
    i += 1"""

Output_df_BIS = Output_df[Output_df['full_BIS'] == 0]
Output_df_BIS = Output_df_BIS[['pred_BIS', 'true_BIS', 'case_id']]
Output_df_MAP = Output_df[Output_df['full_MAP'] == 0]
Output_df_MAP = Output_df_MAP[['pred_MAP', 'true_MAP', 'case_id']]

Output_df_BIS = Output_df_BIS[Output_df_BIS['true_BIS'] != 0]
Output_df_MAP = Output_df_MAP[Output_df_MAP['true_MAP'] != 0]

# Patients_test.to_csv("./Patients_test.csv")
# %% Analyse results


compute_metrics(Output_df_BIS)
compute_metrics(Output_df_MAP)
# plot_results(Output_df)

  0%|          | 0/45 [00:00<?, ?it/s]

70
101
218
247
353
405
447
537
545
593
636
663
711
734
751
831
894
926
1047
1213
1237
1613
1662
1690
1994
2157
2379
2382
2409
2527
2528
2975
3050
3070
3073
3376
4050
4091
4098
4172
4177
4202
4212
4277
4678
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-13.3 \pm 28.4$ & $21.4 \pm 23.3$ & $11.3 \pm 3.8$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$24.4 \pm 14.2$ & $28.3 \pm 11.5$ & $27.4 \pm 11.8$


(545.0, 4277.0)

# all_reg

In [ ]:
"""Script to use regression technique in order to fit the pharmacodynamic of the Propofol/Remifentanil effect.

Created on Wed May 18 08:47:45 2022

@author: aubouinb
"""


import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import PredefinedSplit
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet, TheilSenRegressor, BayesianRidge, HuberRegressor, SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
#from metrics_functions import compute_metrics, plot_results, plot_case, plot_surface


# %% Load dataset
Patients_train = pd.read_csv("Patients_train.csv", index_col=0)
Patients_test = pd.read_csv("Patients_test.csv", index_col=0)

# %% Undersample data

step = 60  # Undersampling step


Patients_test_full = Patients_test.copy()

Patients_train_BIS = Patients_train[Patients_train['full_BIS'] == 0]
Patients_test_BIS = Patients_test[Patients_test['full_BIS'] == 0]
Patients_train_MAP = Patients_train[Patients_train['full_MAP'] == 0]
Patients_test_MAP = Patients_test[Patients_test['full_MAP'] == 0]

Patients_train_BIS = Patients_train_BIS[::step]
Patients_test_BIS = Patients_test_BIS[::step]
Patients_train_MAP = Patients_train_MAP[::step]
Patients_test_MAP = Patients_test_MAP[::step]

# %% Model based Regressions

feature = 'All'
cov = ['age', 'sex', 'height', 'weight']
Ce_bis_eleveld = ['Ce_Prop_Eleveld', 'Ce_Rem_Eleveld']
Ce_map_eleveld = ['Ce_Prop_MAP_Eleveld', 'Ce_Rem_MAP_Eleveld']
Cplasma_eleveld = ['Cp_Prop_Eleveld', 'Cp_Rem_Eleveld']


# feat_A
if feature == 'All':
    X_col = cov + ['bmi', 'lbm', 'mean_HR'] + Ce_map_eleveld + Ce_bis_eleveld + Cplasma_eleveld
elif feature == '-bmi':
    X_col = cov + ['lbm', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-lbm':
    X_col = cov + ['bmi', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-map':
    X_col = cov + ['bmi', 'lbm', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-hr':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-Cplasma':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld
elif feature == '-Cmap':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Cplasma_eleveld
elif feature == '-Cbis':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case', 'mean_HR'] + Ce_map_eleveld + Cplasma_eleveld


Patients_train_BIS = Patients_train_BIS[X_col + ['caseid', 'BIS', 'train_set']].dropna()
Patients_test_BIS = Patients_test_BIS[X_col + ['caseid', 'BIS', 'Time']].dropna()
Patients_train_MAP = Patients_train_MAP[X_col + ['caseid', 'MAP', 'train_set']].dropna()
Patients_test_MAP = Patients_test_MAP[X_col + ['caseid', 'MAP', 'Time']].dropna()


for name_rg in tqdm(['SVR', 'ElasticNet', 'KNeighborsRegressor', 'KernelRidge']):
    print(name_rg+"..................................")
    filename = 'reg_' + name_rg + '_feat_' + feature + '.pkl'
    poly_degree = 1
    pca_bool = False
    regressors = {}

    try:
        results_BIS = pd.read_csv("results_BIS.csv", index_col=0)
        results_MAP = pd.read_csv("results_MAP.csv", index_col=0)
    except:
        results_BIS = Patients_test_BIS[['Time', 'caseid', 'BIS']].copy()
        results_MAP = Patients_test_MAP[['Time', 'caseid', 'MAP']].copy()

    Train_data_BIS = pd.DataFrame()
    Test_data_BIS = pd.DataFrame()
    Train_data_BIS['case_id'] = Patients_train_BIS['caseid']
    Test_data_BIS['case_id'] = Patients_test_BIS['caseid']

    Train_data_MAP = pd.DataFrame()
    Test_data_MAP = pd.DataFrame()
    Train_data_MAP['case_id'] = Patients_train_MAP['caseid']
    Test_data_MAP['case_id'] = Patients_test_MAP['caseid']

    i = 0

    for y_col in ['BIS', 'MAP']:
        # --------------Training-------------
        if y_col == 'BIS':
            Patients_train = Patients_train_BIS
            Patients_test = Patients_test_BIS
        elif y_col == 'MAP':
            Patients_train = Patients_train_MAP
            Patients_test = Patients_test_MAP
        try:  # Try to load trained regressor
            regressors = pickle.load(open(filename, 'rb'))
            rg = regressors[y_col]
            print("load ok")

        except:  # Otherwhise train the regressors and save it
            Y_train = Patients_train[y_col]
            if name_rg in ['ElasticNet', 'TheilSenRegressor', 'BayesianRidge',
                           'KNeighborsRegressor', 'HuberRegressor', 'SGDRegressor']:
                X_train = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                    Patients_train[X_col].values)
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                if pca_bool:
                    pca = PCA()  # n_components=4)
                    X_train = pca.fit_transform(X_train)
                    plt.plot(pca.explained_variance_ratio_, '-o')
                    X_train = X_train[:, 0:20]
            elif name_rg == 'KernelRidge' or name_rg == 'SVR' or name_rg == 'MLPRegressor':
                X_train = Patients_train[X_col].values
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)

            ps = PredefinedSplit(Patients_train['train_set'].values)
            # ---ElasticNet----
            if name_rg == 'ElasticNet':
                rg = ElasticNet(max_iter=100000)
                Gridsearch = GridSearchCV(rg, {'alpha': np.logspace(-4, 0, 5), 'l1_ratio': np.linspace(0, 1, 11)},
                                          n_jobs=8, cv=ps, scoring='r2', verbose=0)
                Gridsearch.fit(X_train, Y_train)

            # ---KernelRidge----
            elif name_rg == 'KernelRidge':
                parameters = {'kernel': ('linear', 'rbf', 'polynomial'), 'alpha': np.logspace(-3, 1, 5)}
                rg = KernelRidge()
                #kmeans = KMeans(n_clusters=5000, random_state=0, verbose=4).fit(np.concatenate((X_train,np.expand_dims(Y_train,axis=1)),axis=1))
                Gridsearch = GridSearchCV(rg, parameters, cv=ps, n_jobs=8,
                                          scoring='neg_mean_squared_error', return_train_score=True, verbose=4)
                Gridsearch.fit(X_train, Y_train)

            # ---SVR----
            elif name_rg == 'SVR':
                rg = SVR(verbose=0, shrinking=False, cache_size=1000)  # kernel = 'poly', 'rbf'; 'linear'
                Gridsearch = GridSearchCV(rg, {'kernel': ['rbf'], 'C': [0.1],
                                               'gamma': np.logspace(-1, 3, 5), 'epsilon': np.logspace(-3, 1, 5)},  # np.logspace(-2,1,3)
                                          n_jobs=8, cv=ps, scoring='r2', verbose=0)

                Gridsearch.fit(X_train[:], Y_train[:])

            elif name_rg == 'MLPRegressor':
                rg = MLPRegressor(verbose=0, learning_rate='adaptive', max_iter=1000, random_state=8)
                Gridsearch = GridSearchCV(rg, {'hidden_layer_sizes': [512, 1024], 'alpha': 10.0 ** -np.arange(1, 4),
                                               'activation': ('tanh', 'relu', 'logistic', 'identity')}, cv=ps, n_jobs=6)  # ,'relu','logistic','identity'v
                if y_col == 'BIS':
                    Gridsearch.fit(X_train, Y_train/100)
                elif y_col == 'MAP':
                    Gridsearch.fit(X_train, Y_train/150)

            elif name_rg == 'TheilSenRegressor':
                rg = TheilSenRegressor()
                Gridsearch = GridSearchCV(rg, {'max_subpopulation': [1e4, 1e5]}, n_jobs=6, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'BayesianRidge':
                rg = BayesianRidge(n_iter=1000)
                Gridsearch = GridSearchCV(rg, {'alpha_1': [1e-6, 1e-7, 1e-5],
                                               'alpha_2': [1e-6, 1e-7, 1e-5], 'lambda_1': [1e-6, 1e-7, 1e-5],
                                               'lambda_2': [1e-6, 1e-7, 1e-5]}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'KNeighborsRegressor':
                rg = KNeighborsRegressor(n_jobs=8)
                Gridsearch = GridSearchCV(
                    rg, {'n_neighbors': [500, 1000, 2000, 3000], 'weights': ('uniform', 'distance')}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'HuberRegressor':
                rg = HuberRegressor(max_iter=1000)
                Gridsearch = GridSearchCV(rg, {'epsilon': [1.2, 1.35, 1.5, 2], 'alpha': [
                                          1e-5, 1e-4, 1e-3]}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'SGDRegressor':
                rg = SGDRegressor()
                Gridsearch = GridSearchCV(rg, {'loss': ('squared_error', 'huber',
                                                        'epsilon_insensitive', 'squared_epsilon_insensitive'),
                                               'alpha': [1e-6, 1e-7, 1e-5]}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            print(Gridsearch.best_params_)

            rg = Gridsearch.best_estimator_
            regressors[y_col] = rg
            pickle.dump(regressors, open(filename, 'wb'))

        # --------------test performances on test cases-------------

        if name_rg in ['ElasticNet', 'TheilSenRegressor', 'BayesianRidge',
                       'KNeighborsRegressor', 'HuberRegressor', 'SGDRegressor']:
            X_train = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                Patients_train[X_col].values)
            scaler = StandardScaler()
            scaler.fit(X_train)
            pca = PCA()  # n_components=4)
            pca.fit(X_train)

            X_test = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                Patients_test[X_col].values)
            X_test = scaler.transform(X_test)
            if pca_bool:
                X_test = pca.transform(X_test)
                X_test = X_test[:, 0:20]

            y_predicted = rg.predict(X_test)

        elif name_rg == 'KernelRidge' or name_rg == 'SVR':
            X_train = Patients_train[X_col].values
            scaler = StandardScaler()
            scaler.fit(X_train)

            X_test = Patients_test[X_col].values
            X_test = scaler.transform(X_test)
            y_predicted = rg.predict(X_test)

        elif name_rg == 'MLPRegressor':
            X_train = Patients_train[X_col].values
            scaler = StandardScaler()
            scaler.fit(X_train)

            X_test = Patients_test[X_col].values
            X_test = scaler.transform(X_test)

            if y_col == 'BIS':
                y_predicted = rg.predict(X_test)*100
            elif y_col == 'MAP':
                y_predicted = rg.predict(X_test)*150

        col_name = 'pred_' + y_col + '_' + name_rg
        if y_col == 'BIS':
            Test_data_BIS['true_' + y_col] = Patients_test[y_col]
            Test_data_BIS['pred_' + y_col] = y_predicted
            results_BIS.loc[:, col_name] = y_predicted
        else:
            Test_data_MAP['true_' + y_col] = Patients_test[y_col]
            Test_data_MAP['pred_' + y_col] = y_predicted
            results_MAP.loc[:, col_name] = y_predicted
        # -----------------test performances on train cases--------------------

        if name_rg in ['ElasticNet', 'TheilSenRegressor', 'BayesianRidge',
                       'KNeighborsRegressor', 'HuberRegressor', 'SGDRegressor']:
            X_train = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                Patients_train[X_col].values)
            X_train = scaler.transform(X_train)
            if pca_bool:
                X_train = pca.transform(X_train)
                X_train = X_train[:, 0:20]

            y_predicted_train = rg.predict(X_train)

        elif name_rg == 'KernelRidge' or name_rg == 'SVR':
            X_train = Patients_train[X_col].values
            X_train = scaler.transform(X_train)

            y_predicted_train = rg.predict(X_train)

        elif name_rg == 'MLPRegressor':
            X_train = Patients_train[X_col].values
            X_train = scaler.transform(X_train)

            if y_col == 'BIS':
                y_predicted_train = rg.predict(X_train)*100
            elif y_col == 'MAP':
                y_predicted_train = rg.predict(X_train)*150

        if y_col == 'BIS':
            Train_data_BIS['true_' + y_col] = Patients_train[y_col]
            Train_data_BIS['pred_' + y_col] = y_predicted_train
        else:
            Train_data_MAP['true_' + y_col] = Patients_train[y_col]
            Train_data_MAP['pred_' + y_col] = y_predicted_train

        # plot_surface(rg, scaler, feature)

    # results_BIS.to_csv("./results_BIS.csv")
    # results_MAP.to_csv("./results_MAP.csv")

    print('     ***-----------------' + name_rg + '-----------------***')
    print("\n                 ------ Test Results ------")
    max_case_bis, min_case_bis = compute_metrics(Test_data_BIS)
    max_case_map, min_case_map = compute_metrics(Test_data_MAP)
    # print("\n\n                 ------ Train Results ------")
    # compute_metrics(Train_data_BIS)
    # compute_metrics(Train_data_MAP)
    # plot_results(Test_data_BIS, Test_data_MAP, Train_data_BIS, Train_data_MAP)

    # plot_case(results_BIS, results_MAP, Patients_test_full, min_case_bis, min_case_map, max_case_bis, max_case_map)

  0%|          | 0/4 [00:00<?, ?it/s]

SVR..................................
{'C': 0.1, 'epsilon': 0.001, 'gamma': 0.1, 'kernel': 'rbf'}
{'C': 0.1, 'epsilon': 10.0, 'gamma': 0.1, 'kernel': 'rbf'}
     ***-----------------SVR-----------------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-2.1 \pm 28.6$ & $14.0 \pm 21.4$ & $10.0 \pm 2.3$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-0.5 \pm 15.6$ & $10.9 \pm 10.7$ & $12.9 \pm 3.4$
ElasticNet..................................
{'alpha': 0.1, 'l1_ratio': 0.7000000000000001}
{'alpha': 0.1, 'l1_ratio': 1.0}
     ***-----------------ElasticNet-----------------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-4.1 \pm 25.7$ & $18.2 \pm 19.1$ & $10.6 \pm 2.7$
pred_MAP
true_MAP
                 ______   MAP results 

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_neighbors': 500, 'weights': 'distance'}


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_neighbors': 1000, 'weights': 'distance'}
     ***-----------------KNeighborsRegressor-----------------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-2.6 \pm 27.8$ & $14.8 \pm 20.5$ & $9.8 \pm 2.2$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$0.1 \pm 15.5$ & $10.7 \pm 10.5$ & $12.8 \pm 3.5$
KernelRidge..................................
Fitting 3 folds for each of 15 candidates, totalling 45 fits


# all features

In [5]:
"""Script to use regression technique in order to fit the pharmacodynamic of the Propofol/Remifentanil effect.

Created on Wed May 18 08:47:45 2022

@author: aubouinb
"""


# %% Load dataset
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import PredefinedSplit
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet, TheilSenRegressor, BayesianRidge, HuberRegressor, SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
#from metrics_functions import compute_metrics, plot_results, plot_case, plot_surface

Patients_train_full = pd.read_csv("Patients_train.csv", index_col=0)
Patients_test_full = pd.read_csv("Patients_test.csv", index_col=0)


# %% Model based Regressions

cov = ['age', 'sex', 'height', 'weight']
Ce_bis_eleveld = ['Ce_Prop_Eleveld', 'Ce_Rem_Eleveld']
Ce_map_eleveld = ['Ce_Prop_MAP_Eleveld', 'Ce_Rem_MAP_Eleveld']
Cplasma_eleveld = ['Cp_Prop_Eleveld', 'Cp_Rem_Eleveld']

for feature in ['All', '-bmi', '-lbm',  '-hr', '-Cplasma', '-Cmap', '-Cbis']:

    step = 60  # Undersampling step

    Patients_train_BIS = Patients_train_full[Patients_train_full['full_BIS'] == 0].copy()
    Patients_test_BIS = Patients_test_full[Patients_test_full['full_BIS'] == 0].copy()
    Patients_train_MAP = Patients_train_full[Patients_train_full['full_MAP'] == 0].copy()
    Patients_test_MAP = Patients_test_full[Patients_test_full['full_MAP'] == 0].copy()

    Patients_train_BIS = Patients_train_BIS[::step]
    Patients_test_BIS = Patients_test_BIS[::step]
    Patients_train_MAP = Patients_train_MAP[::step]
    Patients_test_MAP = Patients_test_MAP[::step]

    # feat_A
    if feature == 'All':
        X_col = cov + ['bmi', 'lbm',  'mean_HR'] + Ce_map_eleveld + Ce_bis_eleveld + Cplasma_eleveld
    elif feature == '-bmi':
        X_col = cov + ['lbm',  'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
    elif feature == '-lbm':
        X_col = cov + ['bmi',  'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
    elif feature == '-map':
        X_col = cov + ['bmi', 'lbm', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
    elif feature == '-hr':
        X_col = cov + ['bmi', 'lbm'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
    elif feature == '-Cplasma':
        X_col = cov + ['bmi', 'lbm',  'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld
    elif feature == '-Cmap':
        X_col = cov + ['bmi', 'lbm',  'mean_HR'] + Ce_bis_eleveld + Cplasma_eleveld
    elif feature == '-Cbis':
        X_col = cov + ['bmi', 'lbm', 'mean_HR'] + Ce_map_eleveld + Cplasma_eleveld

    Patients_train_BIS = Patients_train_BIS[X_col + ['caseid', 'BIS', 'train_set']].dropna()
    Patients_test_BIS = Patients_test_BIS[X_col + ['caseid', 'BIS', 'Time']].dropna()
    Patients_train_MAP = Patients_train_MAP[X_col + ['caseid', 'MAP', 'train_set']].dropna()
    Patients_test_MAP = Patients_test_MAP[X_col + ['caseid', 'MAP', 'Time']].dropna()

    name_rg = 'KNeighborsRegressor'
    filename = 'reg_' + name_rg + '_feat_' + feature + '_test.pkl'
    poly_degree = 1
    pca_bool = False
    regressors = {}

    try:
        results_BIS = pd.read_csv("./results_BIS.csv", index_col=0)
        results_MAP = pd.read_csv("./results_MAP.csv", index_col=0)
    except:
        results_BIS = Patients_test_BIS[['Time', 'caseid', 'BIS']].copy()
        results_MAP = Patients_test_MAP[['Time', 'caseid', 'MAP']].copy()

    Train_data_BIS = pd.DataFrame()
    Test_data_BIS = pd.DataFrame()
    Train_data_BIS['case_id'] = Patients_train_BIS['caseid']
    Test_data_BIS['case_id'] = Patients_test_BIS['caseid']

    Train_data_MAP = pd.DataFrame()
    Test_data_MAP = pd.DataFrame()
    Train_data_MAP['case_id'] = Patients_train_MAP['caseid']
    Test_data_MAP['case_id'] = Patients_test_MAP['caseid']

    i = 0

    for y_col in ['BIS', 'MAP']:
        if y_col == 'BIS':
            Patients_train = Patients_train_BIS
            Patients_test = Patients_test_BIS
        elif y_col == 'MAP':
            Patients_train = Patients_train_MAP
            Patients_test = Patients_test_MAP
        # --------------Training-------------

        try:  # Try to load trained regressor
            regressors = pickle.load(open(filename, 'rb'))
            rg = regressors[y_col]
            print("load ok")

        except:  # Otherwhise train the regressors and save it
            Y_train = Patients_train[y_col]
            if name_rg in ['ElasticNet', 'TheilSenRegressor', 'BayesianRidge',
                           'KNeighborsRegressor', 'HuberRegressor', 'SGDRegressor']:
                X_train = Patients_train[X_col]
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)

            elif name_rg == 'KernelRidge' or name_rg == 'SVR' or name_rg == 'MLPRegressor':
                X_train = Patients_train[X_col].values
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)

            ps = PredefinedSplit(Patients_train['train_set'].values)
            # ---ElasticNet----
            if name_rg == 'ElasticNet':
                rg = ElasticNet(max_iter=100000)
                Gridsearch = GridSearchCV(rg, {'alpha': np.logspace(-4, 0, 5), 'l1_ratio': np.linspace(0, 1, 11)},
                                          n_jobs=8, cv=ps, scoring='r2', verbose=0)
                Gridsearch.fit(X_train, Y_train)

            # ---KernelRidge----
            elif name_rg == 'KernelRidge':
                parameters = {'kernel': ('linear', 'rbf', 'polynomial'), 'alpha': np.logspace(-3, 1, 5)}
                rg = KernelRidge()
                #kmeans = KMeans(n_clusters=5000, random_state=0, verbose=4).fit(np.concatenate((X_train,np.expand_dims(Y_train,axis=1)),axis=1))
                Gridsearch = GridSearchCV(rg, parameters, cv=ps, n_jobs=8,
                                          scoring='neg_mean_squared_error', return_train_score=True, verbose=4)
                Gridsearch.fit(X_train, Y_train)

            # ---SVR----
            elif name_rg == 'SVR':
                rg = SVR(verbose=0, shrinking=False, cache_size=1000)  # kernel = 'poly', 'rbf'; 'linear'
                Gridsearch = GridSearchCV(rg, {'kernel': ['rbf'], 'C': [0.1],
                                               'gamma': np.logspace(-1, 3, 5), 'epsilon': np.logspace(-3, 1, 5)},  # np.logspace(-2,1,3)
                                          n_jobs=8, cv=ps, scoring='r2', verbose=0)

                Gridsearch.fit(X_train[:], Y_train[:])

            elif name_rg == 'MLPRegressor':
                rg = MLPRegressor(verbose=0, learning_rate='adaptive', max_iter=1000, random_state=8)
                Gridsearch = GridSearchCV(rg, {'hidden_layer_sizes': [512, 1024], 'alpha': 10.0 ** -np.arange(1, 4),
                                               'activation': ('tanh', 'relu', 'logistic', 'identity')}, cv=ps, n_jobs=6)  # ,'relu','logistic','identity'v
                if y_col == 'BIS':
                    Gridsearch.fit(X_train, Y_train/100)
                elif y_col == 'MAP':
                    Gridsearch.fit(X_train, Y_train/150)

            elif name_rg == 'TheilSenRegressor':
                rg = TheilSenRegressor()
                Gridsearch = GridSearchCV(rg, {'max_subpopulation': [1e4, 1e5]}, n_jobs=6, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'BayesianRidge':
                rg = BayesianRidge(n_iter=1000)
                Gridsearch = GridSearchCV(rg, {'alpha_1': [1e-6, 1e-7, 1e-5], 'alpha_2': [1e-6, 1e-7, 1e-5],
                                               'lambda_1': [1e-6, 1e-7, 1e-5], 'lambda_2': [1e-6, 1e-7, 1e-5]},
                                          n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'KNeighborsRegressor':
                rg = KNeighborsRegressor(n_jobs=8)
                Gridsearch = GridSearchCV(
                    rg, {'n_neighbors': [500, 1000, 2000, 3000], 'weights': ('uniform', 'distance')}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'HuberRegressor':
                rg = HuberRegressor(max_iter=1000)
                Gridsearch = GridSearchCV(rg, {'epsilon': [1.2, 1.35, 1.5, 2], 'alpha': [
                                          1e-5, 1e-4, 1e-3]}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            elif name_rg == 'SGDRegressor':
                rg = SGDRegressor()
                Gridsearch = GridSearchCV(rg, {'loss': ('squared_error', 'huber',
                                                        'epsilon_insensitive', 'squared_epsilon_insensitive'),
                                               'alpha': [1e-6, 1e-7, 1e-5]}, n_jobs=8, cv=ps)
                Gridsearch.fit(X_train, Y_train)
            print(Gridsearch.best_params_)

            rg = Gridsearch.best_estimator_
            regressors[y_col] = rg
            pickle.dump(regressors, open(filename, 'wb'))

        # --------------test performances on test cases-------------

        if name_rg in ['ElasticNet', 'TheilSenRegressor', 'BayesianRidge',
                       'KNeighborsRegressor', 'HuberRegressor', 'SGDRegressor']:
            X_train = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                Patients_train[X_col].values)
            scaler = StandardScaler()
            scaler.fit(X_train)
            pca = PCA()  # n_components=4)
            pca.fit(X_train)

            X_test = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                Patients_test[X_col].values)
            X_test = scaler.transform(X_test)
            if pca_bool:
                X_test = pca.transform(X_test)
                X_test = X_test[:, 0:20]

            y_predicted = rg.predict(X_test)

        elif name_rg == 'KernelRidge' or name_rg == 'SVR':
            X_train = Patients_train[X_col].values
            scaler = StandardScaler()
            scaler.fit(X_train)

            X_test = Patients_test[X_col].values
            X_test = scaler.transform(X_test)
            y_predicted = rg.predict(X_test)

        elif name_rg == 'MLPRegressor':
            X_train = Patients_train[X_col].values
            scaler = StandardScaler()
            scaler.fit(X_train)

            X_test = Patients_test[X_col].values
            X_test = scaler.transform(X_test)

            if y_col == 'BIS':
                y_predicted = rg.predict(X_test)*100
            elif y_col == 'MAP':
                y_predicted = rg.predict(X_test)*150

        col_name = 'pred_' + y_col + '_' + name_rg
        if y_col == 'BIS':
            Test_data_BIS['true_' + y_col] = Patients_test[y_col]
            Test_data_BIS['pred_' + y_col] = y_predicted
            results_BIS.loc[:, col_name] = y_predicted
        else:
            Test_data_MAP['true_' + y_col] = Patients_test[y_col]
            Test_data_MAP['pred_' + y_col] = y_predicted
            results_MAP.loc[:, col_name] = y_predicted
        # -----------------test performances on train cases--------------------

        if name_rg in ['ElasticNet', 'TheilSenRegressor', 'BayesianRidge',
                       'KNeighborsRegressor', 'HuberRegressor', 'SGDRegressor']:
            X_train = PolynomialFeatures(degree=poly_degree, include_bias=False).fit_transform(
                Patients_train[X_col].values)
            X_train = scaler.transform(X_train)
            if pca_bool:
                X_train = pca.transform(X_train)
                X_train = X_train[:, 0:20]

            y_predicted_train = rg.predict(X_train)

        elif name_rg == 'KernelRidge' or name_rg == 'SVR':
            X_train = Patients_train[X_col].values
            X_train = scaler.transform(X_train)

            y_predicted_train = rg.predict(X_train)

        elif name_rg == 'MLPRegressor':
            X_train = Patients_train[X_col].values
            X_train = scaler.transform(X_train)

            if y_col == 'BIS':
                y_predicted_train = rg.predict(X_train)*100
            elif y_col == 'MAP':
                y_predicted_train = rg.predict(X_train)*150

        if y_col == 'BIS':
            Train_data_BIS['true_' + y_col] = Patients_train[y_col]
            Train_data_BIS['pred_' + y_col] = y_predicted_train
        else:
            Train_data_MAP['true_' + y_col] = Patients_train[y_col]
            Train_data_MAP['pred_' + y_col] = y_predicted_train

        # plot_surface(rg, scaler, feature)

    # results_BIS.to_csv("./results_BIS.csv")
    # results_MAP.to_csv("./results_MAP.csv")

    print('     ***----------' + name_rg + ' ' + feature + '----------***')
    print("\n                 ------ Test Results ------")
    max_case_bis, min_case_bis = compute_metrics(Test_data_BIS)
    max_case_map, min_case_map = compute_metrics(Test_data_MAP)
    # print("\n\n                 ------ Train Results ------")
    # compute_metrics(Train_data_BIS)
    # compute_metrics(Train_data_MAP)
    # plot_results(Test_data_BIS, Test_data_MAP, Train_data_BIS, Train_data_MAP)

    # plot_case(results_BIS, results_MAP, Patients_test_full, min_case_bis, min_case_map, max_case_bis, max_case_map)

load ok
load ok
     ***----------KNeighborsRegressor All----------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-2.6 \pm 27.8$ & $14.8 \pm 20.5$ & $9.8 \pm 2.2$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$0.1 \pm 15.5$ & $10.7 \pm 10.5$ & $12.8 \pm 3.5$


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_neighbors': 500, 'weights': 'distance'}


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_neighbors': 1000, 'weights': 'distance'}
     ***----------KNeighborsRegressor -bmi----------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-2.6 \pm 27.5$ & $15.0 \pm 20.5$ & $9.8 \pm 2.1$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$0.0 \pm 15.5$ & $10.6 \pm 10.6$ & $12.7 \pm 3.4$


...

{'n_neighbors': 1000, 'weights': 'distance'}
     ***----------KNeighborsRegressor -Cmap----------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-3.3 \pm 28.5$ & $15.1 \pm 20.9$ & $10.1 \pm 2.2$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$0.3 \pm 15.5$ & $10.7 \pm 10.5$ & $12.8 \pm 3.5$


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_neighbors': 500, 'weights': 'distance'}


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_neighbors': 1000, 'weights': 'distance'}
     ***----------KNeighborsRegressor -Cbis----------***

                 ------ Test Results ------
pred_BIS
true_BIS
                 ______   BIS results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$-2.5 \pm 28.0$ & $15.0 \pm 20.5$ & $10.0 \pm 2.2$
pred_MAP
true_MAP
                 ______   MAP results   ______
     MDPE 	 & 	 MDAPE 	 & 	 RMSE       
$0.4 \pm 15.5$ & $10.7 \pm 10.5$ & $12.9 \pm 3.6$
